In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2023-04-25 14:51:52.294278: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 14:51:52.319549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


In [3]:
import shutil
src_path = r"/home/fish/programing/cancerClassifier/patch_images/"
dst_path = r"/home/fish/programing/cancerClassifier/splitData/"

# split the dataset 
# dont runn unless you wanna split the data again (you dont)
with open('data_labels_mainData.csv', 'r') as file:
    
    for line in file.readlines()[1:]:
        li = line.split(',')
        shutil.copyfile(src_path + str(li[2]), dst_path + str(li[3]) + '/' + str(li[2]))
        print(li)
        



['22405', '1', '22405.png', 'fibroblast', '0', '0\n']
['22406', '1', '22406.png', 'fibroblast', '0', '0\n']
['22407', '1', '22407.png', 'fibroblast', '0', '0\n']
['22408', '1', '22408.png', 'fibroblast', '0', '0\n']
['22409', '1', '22409.png', 'fibroblast', '0', '0\n']
['22410', '1', '22410.png', 'fibroblast', '0', '0\n']
['22411', '1', '22411.png', 'fibroblast', '0', '0\n']
['22412', '1', '22412.png', 'fibroblast', '0', '0\n']
['22413', '1', '22413.png', 'fibroblast', '0', '0\n']
['22414', '1', '22414.png', 'fibroblast', '0', '0\n']
['22415', '1', '22415.png', 'fibroblast', '0', '0\n']
['22417', '1', '22417.png', 'inflammatory', '1', '0\n']
['22418', '1', '22418.png', 'inflammatory', '1', '0\n']
['22419', '1', '22419.png', 'inflammatory', '1', '0\n']
['22420', '1', '22420.png', 'inflammatory', '1', '0\n']
['22421', '1', '22421.png', 'inflammatory', '1', '0\n']
['22422', '1', '22422.png', 'inflammatory', '1', '0\n']
['22423', '1', '22423.png', 'others', '3', '0\n']
['22424', '1', '2242

['17249', '29', '17249.png', 'inflammatory', '1', '0\n']
['17250', '29', '17250.png', 'inflammatory', '1', '0\n']
['17252', '29', '17252.png', 'others', '3', '0\n']
['17253', '29', '17253.png', 'others', '3', '0\n']
['11747', '30', '11747.png', 'inflammatory', '1', '0\n']
['11748', '30', '11748.png', 'inflammatory', '1', '0\n']
['11749', '30', '11749.png', 'others', '3', '0\n']
['11750', '30', '11750.png', 'others', '3', '0\n']
['11751', '30', '11751.png', 'others', '3', '0\n']
['11752', '30', '11752.png', 'others', '3', '0\n']
['11753', '30', '11753.png', 'others', '3', '0\n']
['11754', '30', '11754.png', 'others', '3', '0\n']
['11755', '30', '11755.png', 'others', '3', '0\n']
['11756', '30', '11756.png', 'others', '3', '0\n']
['11757', '30', '11757.png', 'others', '3', '0\n']
['11758', '30', '11758.png', 'others', '3', '0\n']
['11759', '30', '11759.png', 'others', '3', '0\n']
['11760', '30', '11760.png', 'others', '3', '0\n']
['11761', '30', '11761.png', 'others', '3', '0\n']
['11762

In [32]:
import pathlib
dataset_url = "file:///home/fish/programing/cancerClassifier/splitData.tar.gz"
data_dir = tf.keras.utils.get_file('splitData', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)



In [34]:
batch_size = 32
img_height = 27
img_width = 27

train_ds = tf.keras.utils.image_dataset_from_directory(
  '/home/fish/programing/cancerClassifier/splitData/',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  '/home/fish/programing/cancerClassifier/splitData/',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 9896 files belonging to 4 classes.
Using 7917 files for training.
Found 9896 files belonging to 4 classes.
Using 1979 files for validation.


In [35]:
class_names = train_ds.class_names
print(class_names)


['epithelial', 'fibroblast', 'inflammatory', 'others']
